

<img src="assets/uk_railways.jpg" alt="Prototype 3" width="600" height="200">

# UK Railways Ticket Price Dashboard
DTSA5304

**Stanislav Liashkov**  2024.07.05


## Introduction

This project starts with the basic exploratory data analysis in order to see what kind of data we have and what dimensions
are potentially to be the most interesting for *Railway management and stakeholders*. The **goal** of this project is to *design and evaluate a dashboard for monitoring ticket price distribution for different trips*. In this project, **visual task** that we optimize our dashboard for is "*to discover bussiness-valuable inisghts from data related to ticket price, classes of passengers and payment methods and to be able to quickly get the summary picture for any route*". 

In order to achive the goal and help users complete *visual task*, I have prepared a few **Low-fidelity prototypes** of dashboard and select the most suitable one. Once I decided which prototype should be taken, I started to design the dashboard with *more details* and finally implemented the dashboard using **Altair and Streamlit**.



## Data

The dataset comes from **Kaggle** ( https://www.kaggle.com/datasets/motsimaslam/national-rail-uk-train-ticket-data) and contains data about **UK Railways Train Tickets** such as *ticket price, type of ticket, payment method, passenger class* etc, **spanning from January to April 2024**. It includes detailed information on various aspects of train journeys, providing insights into the travel patterns and pricing within this period. The dataset encompasses the following key attributes:

- **Ticket Type**: The category of the ticket, such as single, return, or season.
- **Journey Date & Time**: The date and time for each train journey, including both departure and arrival times.
- **Departure Station**: The station from which the journey originates.
- **Arrival Station**: The station at which the journey concludes.
- **Ticket Price**: The cost of the ticket in GBP.
- **Other Details**: Additional information that might include passenger class, train service provider, booking reference, and any applicable discounts or offers.





In [2]:
import pandas as pd

df = pd.read_csv("railway.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31653 entries, 0 to 31652
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Transaction ID       31653 non-null  object
 1   Date of Purchase     31653 non-null  object
 2   Time of Purchase     31653 non-null  object
 3   Purchase Type        31653 non-null  object
 4   Payment Method       31653 non-null  object
 5   Railcard             10735 non-null  object
 6   Ticket Class         31653 non-null  object
 7   Ticket Type          31653 non-null  object
 8   Price                31653 non-null  int64 
 9   Departure Station    31653 non-null  object
 10  Arrival Destination  31653 non-null  object
 11  Date of Journey      31653 non-null  object
 12  Departure Time       31653 non-null  object
 13  Arrival Time         31653 non-null  object
 14  Actual Arrival Time  29773 non-null  object
 15  Journey Status       31653 non-null  object
 16  Reas

The **key dimensions** for our defined **visual task** are going to be:
- Ticket Price
- Purchase Type
- Ticket Class
- Ticket Type
- Source & Destination

## Sketching prototypes and Designing dashboard

### Low-fidelity Prototypes
<figure>
  <img src="assets/prototype1.jpeg" alt="Prototype 1" width="500" height="300">
  <figcaption><b>Prototype 1:</b> Blue colors, barcharts </figcaption>
</figure>

<figure>
  <img src="assets/prototype2.jpeg" alt="Prototype 2" width="500" height="300">
  <figcaption><b>Prototype 2:</b> Green colors, smooth histogram + pies</figcaption>
</figure>

<figure>
  <img src="assets/prototype3.jpeg" alt="Prototype 3" width="500" height="300">
  <figcaption><b>Prototype 3:</b> Yellow colors, binned histogram + pies</figcaption>
</figure>



#### Discussion and selection
I've come up with 3 slightly different sketches that can potentially become a prototype for our dashboard. They all share similar goals to show stakeholders **distribution of ticket price along with additional categorical data (Ticket Class and Type, Purchase Type) and option to select any Source and Destination.** The sketched dashboard differ in coloring, positions of plots and type of plots. Let's discuss which prototype should be chosen.

Among all prototypes, I would like to choose **the third (yellow) one** for several reasons:
- Yellow/Golden color fits better as it is associated with money or value (in my opinion)
- Average manager likely finds it simpler to interpret binned discrete histogram rather than density plot
- Pie charts fit better than barplots because it is easier to see what category in what fraction contribute to the whole
- Select source and select destination buttons are positioned in more natural way (from top to bottom)

I have to point out that in any case, our selected prototype is likely not perfect and have its disadvantages, though for now, we consider this prototype as a good starting point. One additional improvement for this prototype might be some indicator that shows the **average ticker price** because it is likely an important information (It can be inferred from histogram, but you need to put a bit of effort).

Now, it is time to get to implementation of the dashboard.

https://www.youtube.com/watch?v=pWxDxhWXJos

In [3]:
df.columns

Index(['Transaction ID', 'Date of Purchase', 'Time of Purchase',
       'Purchase Type', 'Payment Method', 'Railcard', 'Ticket Class',
       'Ticket Type', 'Price', 'Departure Station', 'Arrival Destination',
       'Date of Journey', 'Departure Time', 'Arrival Time',
       'Actual Arrival Time', 'Journey Status', 'Reason for Delay',
       'Refund Request'],
      dtype='object')

## TODO
1) Add 5W vector that describes task and answer WHO, WHAT, WHY etc questions
2) Filter Destinations with very low number of trips. Leave only most popular
3) Choose a prototype
4) Implement dashboard
   


In [6]:
df["route"] = df["Departure Station"] +  " -> "+ df["Arrival Destination"]

,route,Transaction ID
40,Manchester Piccadilly -> Liverpool Lime Street,4628
24,London Euston -> Birmingham New Street,4209
30,London Kings Cross -> York,3922
35,London Paddington -> Reading,3873
36,London St Pancras -> Birmingham New Street,3471
...,...,...
60,York -> Edinburgh Waverley,15
47,Manchester Piccadilly -> Warrington,15
62,York -> Liverpool Lime Street,15
64,York -> Wakefield,15


In [24]:
df.groupby("route").count()["Transaction ID"].reset_index().sort_values(by="Transaction ID")[::-1][:8]

,route,Transaction ID
40,Manchester Piccadilly -> Liverpool Lime Street,4628
24,London Euston -> Birmingham New Street,4209
30,London Kings Cross -> York,3922
35,London Paddington -> Reading,3873
36,London St Pancras -> Birmingham New Street,3471
22,Liverpool Lime Street -> Manchester Piccadilly,3002
19,Liverpool Lime Street -> London Euston,1097
25,London Euston -> Manchester Piccadilly,712


In [33]:
df.groupby("Departure Station").count()["Transaction ID"].sort_values()[::-1].index[:5]

Index(['Manchester Piccadilly', 'London Euston', 'Liverpool Lime Street',
       'London Paddington', 'London Kings Cross'],
      dtype='object', name='Departure Station')

In [22]:
set(df["Arrival Destination"].unique().tolist() + df["Departure Station"].unique().tolist())

{'Birmingham New Street',
 'Bristol Temple Meads',
 'Cardiff Central',
 'Coventry',
 'Crewe',
 'Didcot',
 'Doncaster',
 'Durham',
 'Edinburgh',
 'Edinburgh Waverley',
 'Leeds',
 'Leicester',
 'Liverpool Lime Street',
 'London Euston',
 'London Kings Cross',
 'London Paddington',
 'London St Pancras',
 'London Waterloo',
 'Manchester Piccadilly',
 'Nottingham',
 'Nuneaton',
 'Oxford',
 'Peterborough',
 'Reading',
 'Sheffield',
 'Stafford',
 'Swindon',
 'Tamworth',
 'Wakefield',
 'Warrington',
 'Wolverhampton',
 'York'}